In [ ]:
# Importing necessary libraries

import sqlite3 as sql
import kagglehub
import os

In [ ]:
# Creating the path to access the kaggle dataset

path = kagglehub.dataset_download("luizpaulodeoliveira/imdb-project-sql")

# To show what files are stored in the kaggle dataset
print(os.listdir(path))

In [ ]:
# Access only the necessary database file (ie. 'movies.sqlite')
db_file = os.path.join(path, "movies.sqlite")

# Connect to the SQLite database
conn = sql.connect(db_file)
cursor = conn.cursor()

# List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)

In [ ]:
# Only the tables 'director' and 'movies' will be necessary

In [ ]:
# To preview table 'directors'

cursor.execute("SELECT * FROM directors LIMIT 5;")
col_d = [desc[0] for desc in cursor.description]
row_d = cursor.fetchall()

# Print column headers
print(" | ".join(col_d))
print("-" * (len(col_d) * 15))  # Separator

# Print rows
for row in row_d:
    print(" | ".join(str(col) for col in row))

In [ ]:
# To preview table 'movies'

cursor.execute("SELECT * FROM movies LIMIT 5;")
col_m = [desc[0] for desc in cursor.description]
row_m = cursor.fetchall()

# Print column headers
print(" | ".join(col_m))
print("-" * (len(col_m) * 15))  # Separator

# Print rows
for row in row_m:
    print(" | ".join(str(col) for col in row))

In [ ]:
### TASK ###
### Identify the top 10 highest-rated movies (by IMDb rating).

# Determine the top 10 highest-rated movies by:
# a. highest to lowest vote average; and
# b. highest to lowest vote count (if movies have the same vote average)

top_10 = """
SELECT id AS movie_id, title, vote_average, vote_count
FROM movies
ORDER BY vote_average DESC, vote_count DESC
LIMIT 10
"""

# Execute query
cursor.execute(top_10)
header_top_10 = [desc[0] for desc in cursor.description]
results_top_10 = cursor.fetchall()

# Print column headers
print(" | ".join(header_top_10))
print("-" * (len(header_top_10) * 15))  # Separator

# Print rows
for movie in results_top_10:
    print(" | ".join(str(col) for col in movie))

In [ ]:
# IMDb ranks movies based on the average vote received.
# To identify the top 10 highest-rated movies, the list of movies must first be ordered by highest to lowest vote average.
# In the event two or more movies have the same vote average, they will be ordered by highest to lowest vote count (to prioritize movies with more votes).
# Since we only want the top 10 highest-rated movies, the top 10 rows of the dataset will be sufficient.

In [ ]:
### FINDINGS ###

# A high-ranking movie generally has an average vote of 8 and above.
 
# The movies "Stiff Upper Lips", "Dancer, Texas Pop. 81", "Sardaarji", "One Man's Hero", "There Goes My Baby" and "The Prisoner of Zenda"
# are not good representations of a high-ranking movie as their vote count is less than 20, which is significantly lower than the vote count
# of the other movies in the top 10 list.

# A high vote average alone is unable to determine if the movie is popular and successful. 

In [ ]:
### TASK ###
### Find the most prolific directors (directors with the most movies).

# We define a prolific director as one who has directed 20 or more movies

# Count the number of movies directed per director
# Filter only the director IDs with 20 or more movies
# Create a new table for this data

pro_d_id = """
CREATE TABLE d_count AS
SELECT director_id, COUNT(*) AS movie_count
FROM movies
GROUP BY director_id
HAVING COUNT(*) >= 20
ORDER BY movie_count DESC
"""

# Execute query
cursor.execute(pro_d_id)


# Establish the relationship between director ID and director name

pro_d_name = """
SELECT d_count.director_id, 
       directors.name, 
       d_count.movie_count
FROM d_count
JOIN directors ON d_count.director_id = directors.id
"""

# Execute query
cursor.execute(pro_d_name)
header_pro_d = [desc[0] for desc in cursor.description]
results_pro_d = cursor.fetchall()

# Print column headers
print(" | ".join(header_pro_d))
print("-" * (len(header_pro_d) * 15))  # Separator

# Print rows
for d in results_pro_d:
    print(" | ".join(str(col) for col in d))

In [ ]:
# We first define a prolific director as one who has directed 20 or more movies.
# To identify prolific directors, we have to count the number of movies each director has directed based on their ID.
# We only extract the rows with director IDs that have a count of 20 or more
# To find their names, we create a relationship with our list of directors to find their names.

In [ ]:
### FINDINGS ###

# Steven Spielberg can be said to be the most profilic, with a significant difference of movie count (count of 7)
# between himself and the others.

# A prolific director shoud be determined as one who has directed the most number of films in the span of their career,
# and not just based on the numerical figure of movies they have directed.
# ie. The highest average number of directed movies per year wil be a better determinant.

In [ ]:
### TASK ###
### Determine trends in average movie ratings over the years.

# Group the movies by category "year"
# Calculate the average of all the vote_average movie ratings for each year
# List down in chronological order to analyse the trend

ratings_year = """
SELECT
  strftime('%Y', release_date) AS year,
  ROUND(AVG(vote_average), 2) AS vote_average_year
FROM movies
GROUP BY year
ORDER BY year
"""

# Execute query
cursor.execute(ratings_year)
header_ratings = [desc[0] for desc in cursor.description]
results_ratings = cursor.fetchall()

# Print column headers
print(" | ".join(header_ratings))
print("-" * (len(header_ratings) * 15))  # Separator

# Print rows
for row in results_ratings:
    print(" | ".join(str(col) for col in row))

In [ ]:
# We are only interested in the vote average of the all the movies across the years.
# A good measure to use would be to find the average of the vote average for each year.

In [ ]:
### FINDINGS ###

# The trend for movie ratings can be considered quite consistent over the years (ie. generally hovering between 6.5 to 7.3).

# The year 1927 is shown to have the highest average vote across the years.

# There was a small dip in average ratings in the year 1933 (rating of 5.6).

# There was a slight downward trend in ratings starting from year 1977 onwards until year 2016.
# However, the average rating picked up right after in year 2017 (rating of 7.4).

In [ ]:
# Note: Kaggle dataset does not reflect movie genres and actors

In [ ]:
# To check what tables currently exist in the database

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)

In [ ]:
# Remove new tables if necessary, to revert database to original form

cursor.execute("DROP TABLE d_count")

In [ ]:
# To check what tables currently exist in the database after removing tables

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)

In [ ]:
# Close the connection to the SQLite database
conn.close()